# Recursive Cte Lineage

**Example: Recursive CTE (Common Table Expression) Support for Column Lineage**


Demonstrates how clgraph tracks column lineage through recursive CTEs:
- Hierarchical data (org charts, category trees)
- Graph traversal (path finding)
- Sequence generation
- Running totals

### Imports

In [1]:
from clgraph import JSONExporter, Pipeline, RecursiveLineageBuilder
from clgraph.query_parser import RecursiveQueryParser

### Example 1: Simple Recursive Sequence

In [2]:
print("Example 1: Simple Recursive Sequence")

sql_sequence = """
WITH RECURSIVE nums AS (
    SELECT 1 AS n
    UNION ALL
    SELECT n + 1 FROM nums WHERE n < 10
)
SELECT n FROM nums
"""

parser = RecursiveQueryParser(sql_sequence, dialect="postgres")
unit_graph = parser.parse()

print("\nSQL:")
print(sql_sequence)

print("\nQuery Units:")
for unit_id, unit in unit_graph.units.items():
    print(f"  {unit_id}: {unit.unit_type.value}")
    if unit.recursive_cte_info:
        info = unit.recursive_cte_info
        print(f"    - Is Recursive: {info.is_recursive}")
        print(f"    - Union Type: {info.union_type}")
        print(f"    - Base Columns: {info.base_columns}")
        print(f"    - Self Reference Alias: {info.self_reference_alias}")

Example 1: Simple Recursive Sequence

SQL:

WITH RECURSIVE nums AS (
    SELECT 1 AS n
    UNION ALL
    SELECT n + 1 FROM nums WHERE n < 10
)
SELECT n FROM nums


Query Units:
  cte_base:nums_base: cte_base
  cte_recursive:nums_recursive: cte_recursive
  cte:nums: cte
    - Is Recursive: True
    - Union Type: union_all
    - Base Columns: ['n']
    - Self Reference Alias: nums
  main: main_query


### Example 2: Organization Hierarchy

In [3]:
print("Example 2: Organization Hierarchy")

sql_hierarchy = """
WITH RECURSIVE org_hierarchy AS (
    -- Base case: Top-level employees (no manager)
    SELECT id, name, manager_id, 1 AS level
    FROM employees
    WHERE manager_id IS NULL

    UNION ALL

    -- Recursive case: Employees with managers
    SELECT e.id, e.name, e.manager_id, h.level + 1
    FROM employees e
    JOIN org_hierarchy h ON e.manager_id = h.id
)
SELECT id, name, level FROM org_hierarchy
"""

parser2 = RecursiveQueryParser(sql_hierarchy, dialect="postgres")
unit_graph2 = parser2.parse()

print("\nSQL:")
print(sql_hierarchy)

# Find the recursive CTE
cte_unit = unit_graph2.get_unit_by_name("org_hierarchy")
if cte_unit and cte_unit.recursive_cte_info:
    info = cte_unit.recursive_cte_info
    print("\nRecursive CTE Analysis:")
    print(f"  CTE Name: {info.cte_name}")
    print(f"  Self Reference Alias: {info.self_reference_alias}")
    print(f"  Join Condition: {info.join_condition}")
    print(f"  Base Columns: {info.base_columns}")
    print(f"  Recursive Columns: {info.recursive_columns}")

Example 2: Organization Hierarchy

SQL:

WITH RECURSIVE org_hierarchy AS (
    -- Base case: Top-level employees (no manager)
    SELECT id, name, manager_id, 1 AS level
    FROM employees
    WHERE manager_id IS NULL

    UNION ALL

    -- Recursive case: Employees with managers
    SELECT e.id, e.name, e.manager_id, h.level + 1
    FROM employees e
    JOIN org_hierarchy h ON e.manager_id = h.id
)
SELECT id, name, level FROM org_hierarchy


Recursive CTE Analysis:
  CTE Name: org_hierarchy
  Self Reference Alias: h
  Join Condition: e.manager_id = h.id
  Base Columns: ['id', 'name', 'manager_id', 'level']
  Recursive Columns: ['id', 'name', 'manager_id', 'h.level + 1']


### Example 3: Graph Path Traversal

In [4]:
print("Example 3: Graph Path Traversal")

sql_paths = """
WITH RECURSIVE reachable_nodes AS (
    -- Start from node 1
    SELECT target_id AS node_id, 1 AS hops
    FROM edges
    WHERE source_id = 1

    UNION ALL

    -- Follow edges from reachable nodes
    SELECT e.target_id, r.hops + 1
    FROM edges e
    JOIN reachable_nodes r ON e.source_id = r.node_id
    WHERE r.hops < 5
)
SELECT DISTINCT node_id, MIN(hops) AS min_hops
FROM reachable_nodes
GROUP BY node_id
"""

parser3 = RecursiveQueryParser(sql_paths, dialect="postgres")
unit_graph3 = parser3.parse()

print("\nSQL:")
print(sql_paths)

print("\nQuery Structure:")
for unit_id, unit in unit_graph3.units.items():
    unit_type_str = unit.unit_type.value
    if unit.recursive_cte_info:
        unit_type_str += " [RECURSIVE]"
    print(f"  {unit_id}: {unit_type_str}")

Example 3: Graph Path Traversal

SQL:

WITH RECURSIVE reachable_nodes AS (
    -- Start from node 1
    SELECT target_id AS node_id, 1 AS hops
    FROM edges
    WHERE source_id = 1

    UNION ALL

    -- Follow edges from reachable nodes
    SELECT e.target_id, r.hops + 1
    FROM edges e
    JOIN reachable_nodes r ON e.source_id = r.node_id
    WHERE r.hops < 5
)
SELECT DISTINCT node_id, MIN(hops) AS min_hops
FROM reachable_nodes
GROUP BY node_id


Query Structure:
  cte_base:reachable_nodes_base: cte_base
  cte_recursive:reachable_nodes_recursive: cte_recursive
  cte:reachable_nodes: cte [RECURSIVE]
  main: main_query


### Example 4: Path String Accumulation

In [5]:
print("Example 4: Path String Accumulation")

sql_path_string = """
WITH RECURSIVE category_path AS (
    -- Root categories
    SELECT id, name, CAST(name AS VARCHAR(1000)) AS full_path
    FROM categories
    WHERE parent_id IS NULL

    UNION ALL

    -- Child categories
    SELECT c.id, c.name, CONCAT(p.full_path, ' > ', c.name)
    FROM categories c
    JOIN category_path p ON c.parent_id = p.id
)
SELECT id, name, full_path FROM category_path
"""

builder = RecursiveLineageBuilder(sql_path_string, dialect="postgres")
graph = builder.build()

print("\nSQL:")
print(sql_path_string)

print("\nColumn Lineage:")
for name, node in graph.nodes.items():
    if node.layer == "output":
        print(f"  Output: {name}")

Example 4: Path String Accumulation

SQL:

WITH RECURSIVE category_path AS (
    -- Root categories
    SELECT id, name, CAST(name AS VARCHAR(1000)) AS full_path
    FROM categories
    WHERE parent_id IS NULL

    UNION ALL

    -- Child categories
    SELECT c.id, c.name, CONCAT(p.full_path, ' > ', c.name)
    FROM categories c
    JOIN category_path p ON c.parent_id = p.id
)
SELECT id, name, full_path FROM category_path


Column Lineage:
  Output: output.id
  Output: output.name
  Output: output.full_path


### Example 5: Pipeline with Recursive CTE

In [6]:
print("Example 5: Pipeline with Recursive CTE")

queries = [
    (
        "staging",
        "CREATE TABLE staging AS SELECT id, parent_id, name FROM raw_data",
    ),
    (
        "hierarchy",
        """
        CREATE TABLE hierarchy AS
        WITH RECURSIVE tree AS (
            SELECT id, parent_id, name, 1 AS depth
            FROM staging
            WHERE parent_id IS NULL

            UNION ALL

            SELECT s.id, s.parent_id, s.name, t.depth + 1
            FROM staging s
            JOIN tree t ON s.parent_id = t.id
        )
        SELECT * FROM tree
    """,
    ),
]

pipeline = Pipeline(queries, dialect="postgres")

print("\nPipeline Queries:")
for query_id, q in pipeline.table_graph.queries.items():
    print(f"  {query_id}: {q.operation.value}")
    if q.destination_table:
        print(f"    -> {q.destination_table}")

print("\nTable Dependencies:")
for table_name in pipeline.table_graph.tables:
    deps = pipeline.table_graph.get_dependencies(table_name)
    if deps:
        dep_names = [d.table_name for d in deps]
        print(f"  {table_name} <- {dep_names}")

Example 5: Pipeline with Recursive CTE

Pipeline Queries:
  staging: CREATE TABLE
    -> staging
  hierarchy: CREATE TABLE
    -> hierarchy

Table Dependencies:
  staging <- ['raw_data']
  hierarchy <- ['staging', 'tree']


### Example 6: Export with Recursive CTE

In [7]:
print("Example 6: JSON Export with Recursive CTE")

sql_export = """
WITH RECURSIVE nums AS (
    SELECT 1 AS n
    UNION ALL
    SELECT n + 1 FROM nums WHERE n < 5
)
SELECT n * 2 AS doubled FROM nums
"""

export_pipeline = Pipeline([("numbers", sql_export)], dialect="postgres")
exporter = JSONExporter()
export_data = exporter.export(export_pipeline)

print("\nSQL:")
print(sql_export)

print("\nExported Columns:")
for col in export_data.get("columns", []):
    print(f"  {col['table_name']}.{col['column_name']}: {col['node_type']}")

# Summary
print("Summary")
print(
    """
rsive CTE support captures:
tection of WITH RECURSIVE keyword
paration of base case and recursive case
lf-reference alias (e.g., "h" in "JOIN cte h ON ...")
in condition for self-reference
lumn names from both base and recursive cases
ion type (UNION vs UNION ALL)

 metadata is available through:
cursiveQueryParser (query structure analysis)
cursiveLineageBuilder (column lineage analysis)
peline (multi-query analysis)
ON export

rsive CTEs are commonly used for:
erarchical data (org charts, category trees, BOMs)
aph traversal (finding paths, connected components)
quence generation (date ranges, numbers)
nning calculations (running totals, cumulative sums)
"""
)

Example 6: JSON Export with Recursive CTE

SQL:

WITH RECURSIVE nums AS (
    SELECT 1 AS n
    UNION ALL
    SELECT n + 1 FROM nums WHERE n < 5
)
SELECT n * 2 AS doubled FROM nums


Exported Columns:
  nums.n: expression
  nums.: arithmetic
  nums.n: base_column
  numbers_result.doubled: expression
Summary

rsive CTE support captures:
tection of WITH RECURSIVE keyword
paration of base case and recursive case
lf-reference alias (e.g., "h" in "JOIN cte h ON ...")
in condition for self-reference
lumn names from both base and recursive cases
ion type (UNION vs UNION ALL)

 metadata is available through:
cursiveQueryParser (query structure analysis)
cursiveLineageBuilder (column lineage analysis)
peline (multi-query analysis)
ON export

rsive CTEs are commonly used for:
erarchical data (org charts, category trees, BOMs)
aph traversal (finding paths, connected components)
quence generation (date ranges, numbers)
nning calculations (running totals, cumulative sums)



### Visualize Pipeline Lineage

Display the simplified column lineage for recursive CTE pipelines.

In [ ]:
import shutil

from clgraph import visualize_pipeline_lineage

if shutil.which("dot") is None:
    print("⚠️  Graphviz not installed. Install with: brew install graphviz")
else:
    print("Recursive CTE Pipeline - Simplified Lineage:")
    display(visualize_pipeline_lineage(pipeline.column_graph.to_simplified()))

    print("\nExport Pipeline - Simplified Lineage:")
    display(visualize_pipeline_lineage(export_pipeline.column_graph.to_simplified()))